In [ ]:
# app.py (Backend Flask)

import os
import json
from flask import Flask, request, jsonify, render_template
from dotenv import load_dotenv
import google.generativeai as genai

# Carregar variáveis de ambiente do ficheiro .env
load_dotenv()

app = Flask(__name__, template_folder='templates', static_folder='static')

# Configurar a API Key do Gemini
# Certifique-se de que tem a variável GEMINI_API_KEY no seu ficheiro .env
try:
    gemini_api_key = os.environ["GEMINI_API_KEY"]
    if not gemini_api_key:
        raise ValueError("A GEMINI_API_KEY não foi definida no ficheiro .env")
    genai.configure(api_key=gemini_api_key)
    # Modelo Gemini a ser utilizado (pode ajustar conforme necessário)
    model = genai.GenerativeModel('gemini-1.5-flash-latest')
    print("Modelo Gemini configurado com sucesso.")
except Exception as e:
    print(f"Erro crucial ao configurar o Gemini: {e}")
    print("Por favor, verifique se a GEMINI_API_KEY está corretamente definida no seu ficheiro .env.")
    model = None # Impede a execução se o modelo não estiver configurado

DB_FILE = 'event_data_poc.json'

def read_db():
    """Lê os dados do ficheiro JSON."""
    if not os.path.exists(DB_FILE):
        return {"eventos_registados": []}
    try:
        with open(DB_FILE, 'r', encoding='utf-8') as f:
            # Lidar com ficheiro vazio
            content = f.read()
            if not content:
                return {"eventos_registados": []}
            return json.loads(content)
    except (json.JSONDecodeError, FileNotFoundError):
        return {"eventos_registados": []} # Retorna uma estrutura válida em caso de erro

def write_db(data):
    """Escreve os dados no ficheiro JSON."""
    with open(DB_FILE, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

@app.route('/')
def index():
    """Renderiza a página principal do wizard."""
    return render_template('index.html')

@app.route('/api/suggest_themes', methods=['POST'])
def suggest_themes_route():
    """
    Endpoint da API para receber detalhes do evento e devolver sugestões de temas do LLM.
    Atua como o "Orquestrador" que delega ao "Agente de Tematização" (simulado pelo LLM).
    """
    if not model:
        return jsonify({"erro": "O modelo Gemini não está configurado. Verifique a API Key."}), 500

    try:
        event_details = request.json
        if not event_details:
            return jsonify({"erro": "Nenhum detalhe do evento foi fornecido."}), 400

        print(f"Detalhes do evento recebidos para sugestão de temas: {event_details}")

        # --- Lógica do Orquestrador e Prompt para o Agente de Tematização ---
        prompt_parts = [
            "Você é um assistente especialista em criar temas criativos e adequados para eventos corporativos.",
            "Com base nos seguintes detalhes fornecidos pelo organizador, sugira 3 (três) temas distintos para o evento.",
            "Para cada tema, forneça um nome conciso e uma breve descrição (1-2 frases) que explique o conceito.",
            "\nDetalhes do Evento:",
            f"- Nome do Evento: {event_details.get('eventName', 'Não especificado')}",
            f"- Tipo de Evento: {event_details.get('eventType', 'Não especificado')}",
            f"- Número Estimado de Pessoas: {event_details.get('guestCount', 'Não especificado')}",
            f"- Orçamento Preliminar: {event_details.get('budget', 'Não especificado')}",
            f"- Data Desejada: {event_details.get('eventDate', 'Não especificado')}",
            f"- Principal Objetivo/Mensagem: {event_details.get('eventObjective', 'Não especificado')}",
        ]
        if event_details.get('themeIdea'):
            prompt_parts.append(f"- Ideia Inicial de Tema (do organizador): {event_details.get('themeIdea')}")

        prompt_parts.append("\nPor favor, formate a sua resposta claramente, listando cada tema e sua descrição.")
        prompt_parts.append("Exemplo de formato para cada tema:\nTema 1: [Nome do Tema 1]\nDescrição: [Descrição do Tema 1]\n")

        final_prompt = "\n".join(prompt_parts)

        print("\n--- Prompt Estruturado para o Gemini (Agente de Tematização) ---")
        print(final_prompt)
        print("--- Fim do Prompt ---\n")

        # Chamada ao LLM (Gemini)
        try:
            response = model.generate_content(final_prompt)
            # Aceder ao texto da resposta. A API pode variar ligeiramente.
            # Para gemini-1.5-flash, response.text é comum.
            # Verifique a documentação se response.parts[0].text for necessário.
            llm_suggestions_text = response.text
        except Exception as e_gemini:
            print(f"Erro ao comunicar com a API do Gemini: {e_gemini}")
            return jsonify({"erro": f"Não foi possível obter sugestões do LLM: {e_gemini}"}), 503 # Service Unavailable

        print(f"Sugestões recebidas do LLM: {llm_suggestions_text}")

        # Guardar no "banco de dados" JSON
        db_data = read_db()
        novo_evento_registado = {
            "id_evento": f"evt_{len(db_data['eventos_registados']) + 1}",
            "detalhes_fornecidos": event_details,
            "prompt_enviado_llm": final_prompt, # Para debugging e referência
            "sugestoes_temas_llm": llm_suggestions_text
        }
        db_data["eventos_registados"].append(novo_evento_registado)
        write_db(db_data)

        return jsonify({
            "mensagem": "Sugestões de temas geradas com sucesso!",
            "sugestoes_temas": llm_suggestions_text
        })

    except Exception as e:
        print(f"Erro inesperado no endpoint /api/suggest_themes: {e}")
        return jsonify({"erro": f"Ocorreu um erro interno no servidor: {e}"}), 500

if __name__ == '__main__':
    # Garante que as pastas 'templates' e 'static' existem no mesmo nível que app.py
    # ou ajuste os caminhos em Flask(..., template_folder=..., static_folder=...)
    app.run(debug=True, port=5001)
```html
<!DOCTYPE html>
<html lang="pt-PT">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>POC Organizador de Eventos IA</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
</head>
<body>
    <div class="container">
        <header>
            <h1>Organizador de Eventos Corporativos IA</h1>
            <p>Preencha os detalhes abaixo para obter sugestões de temas com IA.</p>
        </header>

        <main>
            <form id="eventWizardForm">
                <fieldset>
                    <legend>Informações Básicas do Evento</legend>

                    <div class="form-group">
                        <label for="eventName">Nome do Evento:</label>
                        <input type="text" id="eventName" name="eventName" placeholder="Ex: Confraternização Anual XPTO" required>
                    </div>

                    <div class="form-group">
                        <label for="eventType">Tipo de Evento:</label>
                        <select id="eventType" name="eventType">
                            <option value="Confraternização">Confraternização</option>
                            <option value="Treinamento">Treinamento</option>
                            <option value="Lançamento de Produto">Lançamento de Produto</option>
                            <option value="Workshop">Workshop</option>
                            <option value="Conferência">Conferência</option>
                            <option value="Team Building">Team Building</option>
                            <option value="Outro">Outro</option>
                        </select>
                    </div>
                </fieldset>

                <fieldset>
                    <legend>Detalhes Adicionais</legend>
                    <div class="form-group">
                        <label for="guestCount">Número Estimado de Pessoas:</label>
                        <input type="number" id="guestCount" name="guestCount" min="1" placeholder="Ex: 50">
                    </div>

                    <div class="form-group">
                        <label for="budget">Orçamento Preliminar (opcional):</label>
                        <input type="text" id="budget" name="budget" placeholder="Ex: Até 5000 EUR">
                    </div>

                    <div class="form-group">
                        <label for="eventDate">Data Desejada (opcional):</label>
                        <input type="date" id="eventDate" name="eventDate">
                    </div>

                    <div class="form-group">
                        <label for="eventObjective">Principal Objetivo/Mensagem do Evento:</label>
                        <textarea id="eventObjective" name="eventObjective" rows="3" placeholder="Ex: Celebrar conquistas, promover integração, lançar novo produto..."></textarea>
                    </div>

                     <div class="form-group">
                        <label for="themeIdea">Alguma ideia inicial de Tema? (Opcional)</label>
                        <input type="text" id="themeIdea" name="themeIdea" placeholder="Ex: Anos 80, Futurista, etc.">
                    </div>
                </fieldset>

                <div class="form-actions">
                    <button type="submit" id="submitButton">Obter Sugestões de Temas</button>
                </div>
            </form>

            <section id="resultsSection" style="display:none;">
                <h2>Sugestões de Temas Geradas pela IA:</h2>
                <div id="loadingIndicator" style="display:none;">
                    <p>A processar com o Gemini... Por favor, aguarde.</p>
                    <div class="spinner"></div>
                </div>
                <div id="errorMessage" class="error-message" style="display:none;"></div>
                <pre id="themeSuggestionsOutput" class="results-output"></pre>
            </section>
        </main>

        <footer>
            <p>POC - Organizador de Eventos com IA & Gemini</p>
        </footer>
    </div>

    <script src="{{ url_for('static', filename='js/script.js') }}"></script>
</body>
</html>
```css
/* static/css/style.css */

body {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    line-height: 1.6;
    margin: 0;
    padding: 0;
    background-color: #f9f9f9;
    color: #333;
    display: flex;
    justify-content: center;
    align-items: flex-start; /* Alinha ao topo para permitir scroll */
    min-height: 100vh;
    padding-top: 20px; /* Espaço no topo */
    padding-bottom: 20px; /* Espaço no fundo */
}

.container {
    background-color: #fff;
    padding: 25px 40px;
    border-radius: 12px;
    box-shadow: 0 8px 25px rgba(0, 0, 0, 0.1);
    width: 100%;
    max-width: 700px; /* Largura máxima do conteúdo */
}

header {
    text-align: center;
    margin-bottom: 30px;
    border-bottom: 1px solid #eee;
    padding-bottom: 20px;
}

header h1 {
    color: #0056b3; /* Azul mais escuro */
    margin-bottom: 5px;
}

header p {
    font-size: 1.1em;
    color: #555;
}

fieldset {
    border: 1px solid #ddd;
    border-radius: 8px;
    padding: 20px;
    margin-bottom: 25px;
}

legend {
    font-weight: bold;
    color: #0056b3;
    padding: 0 10px;
    font-size: 1.2em;
}

.form-group {
    margin-bottom: 20px;
}

.form-group label {
    display: block;
    margin-bottom: 8px;
    font-weight: 600;
    color: #444;
}

.form-group input[type="text"],
.form-group input[type="number"],
.form-group input[type="date"],
.form-group select,
.form-group textarea {
    width: 100%;
    padding: 12px;
    border: 1px solid #ccc;
    border-radius: 6px;
    box-sizing: border-box; /* Importante para padding não aumentar a largura total */
    font-size: 1em;
}

.form-group textarea {
    resize: vertical;
    min-height: 80px;
}

.form-actions {
    text-align: center;
    margin-top: 20px;
}

button[type="submit"] {
    background-color: #007bff; /* Azul primário */
    color: white;
    padding: 12px 25px;
    border: none;
    border-radius: 6px;
    cursor: pointer;
    font-size: 1.1em;
    transition: background-color 0.3s ease;
}

button[type="submit"]:hover {
    background-color: #0056b3; /* Azul mais escuro no hover */
}

#resultsSection {
    margin-top: 30px;
    padding: 20px;
    background-color: #f0f8ff; /* Azul claro */
    border: 1px solid #cce7ff;
    border-radius: 8px;
}

#resultsSection h2 {
    margin-top: 0;
    color: #0056b3;
    border-bottom: 1px solid #cce7ff;
    padding-bottom: 10px;
    margin-bottom: 15px;
}

.results-output {
    white-space: pre-wrap; /* Mantém quebras de linha e espaços */
    font-family: 'Courier New', Courier, monospace;
    background-color: #e9ecef; /* Fundo ligeiramente diferente para o output */
    padding: 15px;
    border-radius: 6px;
    border: 1px solid #ced4da;
    max-height: 400px; /* Altura máxima com scroll */
    overflow-y: auto; /* Scroll vertical se necessário */
}

.error-message {
    color: #d9534f; /* Vermelho para erros */
    background-color: #f2dede;
    border: 1px solid #ebccd1;
    padding: 10px;
    border-radius: 6px;
    margin-bottom: 15px;
}

#loadingIndicator {
    text-align: center;
    padding: 20px;
    color: #0056b3;
}

.spinner {
    border: 4px solid rgba(0, 0, 0, 0.1);
    width: 36px;
    height: 36px;
    border-radius: 50%;
    border-left-color: #007bff;
    margin: 10px auto;
    animation: spin 1s ease infinite;
}

@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}

footer {
    text-align: center;
    margin-top: 40px;
    padding-top: 20px;
    border-top: 1px solid #eee;
    font-size: 0.9em;
    color: #777;
}
```javascript
// static/js/script.js

document.getElementById('eventWizardForm').addEventListener('submit', async function(event) {
    event.preventDefault(); // Impede o envio tradicional do formulário

    const resultsSection = document.getElementById('resultsSection');
    const themeSuggestionsOutput = document.getElementById('themeSuggestionsOutput');
    const loadingIndicator = document.getElementById('loadingIndicator');
    const errorMessageDisplay = document.getElementById('errorMessage');
    const submitButton = document.getElementById('submitButton');

    // Mostrar indicador de carregamento e limpar resultados anteriores
    resultsSection.style.display = 'block';
    loadingIndicator.style.display = 'block';
    themeSuggestionsOutput.textContent = '';
    errorMessageDisplay.style.display = 'none';
    errorMessageDisplay.textContent = '';
    submitButton.disabled = true; // Desabilitar botão durante o pedido

    // Recolher dados do formulário
    const formData = {
        eventName: document.getElementById('eventName').value,
        eventType: document.getElementById('eventType').value,
        guestCount: document.getElementById('guestCount').value,
        budget: document.getElementById('budget').value,
        eventDate: document.getElementById('eventDate').value,
        eventObjective: document.getElementById('eventObjective').value,
        themeIdea: document.getElementById('themeIdea').value
    };

    try {
        const response = await fetch('/api/suggest_themes', {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json',
            },
            body: JSON.stringify(formData),
        });

        loadingIndicator.style.display = 'none'; // Esconder indicador
        const result = await response.json();

        if (response.ok) {
            if (result.sugestoes_temas) {
                themeSuggestionsOutput.textContent = result.sugestoes_temas;
            } else {
                themeSuggestionsOutput.textContent = "Resposta recebida, mas sem sugestões de tema explícitas.";
                console.warn("Resposta do servidor:", result);
            }
        } else {
            errorMessageDisplay.textContent = `Erro ao obter sugestões: ${result.erro || response.statusText}`;
            errorMessageDisplay.style.display = 'block';
            console.error("Erro do servidor:", result);
        }
    } catch (error) {
        loadingIndicator.style.display = 'none'; // Esconder indicador
        errorMessageDisplay.textContent = `Erro de comunicação com o servidor: ${error.message}. Verifique a consola para mais detalhes.`;
        errorMessageDisplay.style.display = 'block';
        console.error('Erro na chamada fetch:', error);
    } finally {
        submitButton.disabled = false; // Reabilitar botão
    }
});

# requirements.txt

Flask
python-dotenv
google-generativeai
```
# .env
# Crie este ficheiro na raiz do seu projeto e adicione a sua API Key do Gemini.
# NÃO partilhe este ficheiro nem o adicione ao controlo de versões (ex: Git).

GEMINI_API_KEY=SUA_API_KEY_DO_GEMINI_AQUI
```
# .gitignore
# Ficheiro para o Git ignorar ficheiros e pastas desnecessários.

# Ambiente Virtual Python
venv/
*.pyc
__pycache__/

# Ficheiro de variáveis de ambiente
.env

# Ficheiros de base de dados (se aplicável, para além da POC)
*.sqlite3
*.db

# Ficheiros de log
*.log

# Ficheiros específicos do OS
.DS_Store
Thumbs.db
```

**Instruções para Executar o Protótipo:**

1.  **Estrutura de Pastas:**
    Crie uma pasta principal para o seu projeto (ex: `poc_organizador_eventos_ia`). Dentro dela, crie:
    * Uma pasta `templates` (e coloque `index.html` dentro dela).
    * Uma pasta `static` e, dentro dela, uma pasta `css` (com `style.css`) e uma pasta `js` (com `script.js`).
    * Coloque `app.py`, `requirements.txt`, `.env` (preenchido) e `.gitignore` na pasta principal do projeto.

2.  **Ambiente Virtual e Dependências:**
    * Abra o terminal/linha de comandos na pasta principal do projeto.
    * Crie um ambiente virtual: `python -m venv venv`
    * Ative o ambiente virtual:
        * Windows: `venv\Scripts\activate`
        * macOS/Linux: `source venv/bin/activate`
    * Instale as dependências: `pip install -r requirements.txt`

3.  **API Key do Gemini:**
    * Certifique-se de que criou o ficheiro `.env` na raiz do projeto e inseriu a sua `GEMINI_API_KEY` válida.

4.  **Executar o Servidor Flask:**
    * No terminal (com o ambiente virtual ativado), execute: `python app.py`
    * O servidor deverá iniciar (por defeito, em `http://127.0.0.1:5001/` ou `http://localhost:5001/` - a porta 5001 foi definida no `app.py` para evitar conflitos com a porta 5000 padrão do Flask).

5.  **Aceder no Navegador:**
    * Abra o seu navegador de internet e vá para `http://127.0.0.1:5001/`.

**Funcionalidades da POC:**

* O formulário em `index.html` permite inserir os detalhes do evento.
* O `script.js` envia esses dados para o endpoint `/api/suggest_themes` no backend Flask.
* O `app.py` (backend):
    * Recebe os dados.
    * Constrói um *prompt* detalhado para o Gemini.
    * Chama a API do Gemini para obter sugestões de temas.
    * Guarda os detalhes do evento e as sugestões num ficheiro `event_data_poc.json` (simulando um banco de dados).
    * Devolve as sugestões ao frontend.
* O `script.js` exibe as sugestões (ou mensagens de erro) na página.

Este protótipo foca-se em demonstrar o fluxo principal de interação com o LLM para uma tarefa específica (sugestão de temas), que é um excelente ponto de partida para a sua apresentação!

Lembre-se que este é um protótipo para demonstração. Para uma aplicação real, seriam necessárias melhorias em segurança, gestão de erros, escalabilidade, um banco de dados mais robusto, e uma arquitetura de agentes mais complexa.

Boa sorte com a sua apresentaç
  ```